In [5]:
import time
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from konlpy.tag import Hannanum, Mecab
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
from kiwipiepy import Kiwi
from pykospacing import Spacing
import re

han = Hannanum()
mec = Mecab()

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = ['_', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

#선어말어미 dictionary
EP = [
    [['ㅈ','ㅓ','_','ㄴ','ㅡ','ㄴ'],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [['ㅇ','ㅓ','_','ㅇ','ㅛ','_'],['ㄷ','ㅏ','_']]
]

#종결 어미 dictionary
EF = [
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ',]],
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [['ㅇ','ㅓ','_','ㅇ','ㅛ','_'],['ㄷ','ㅏ','_']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㅇ','ㅛ'],['']]
    
]

#고유명사 dictionary
NB = [
    
    [['ㅈ','ㅓ'],['ㄴ','ㅏ']],
    [['ㅈ','ㅓ','ㄴ','ㅡ','ㄴ'],['ㄴ','ㅏ','ㄴ','ㅡ','ㄴ']]
    
]
# 보조사
JX = [
   [['ㅇ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']] 
]

XR = [
    [['ㅈ','ㅚ','ㅅ','ㅗ','ㅇ'],['ㅁ','ㅣ','ㅇ','ㅏ','ㄴ'] ]
]

#New function
class JamoDealer:
    jamo = []
    pp = ''
    def __init__(self,lis_word):
        #각 단어들을 받아와서 자모단위로 나눈다.
        print('the jamo starts!')
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
        
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.          
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다. 
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        restored_text = join_jamos(restored_jamo)
        #합쳐진 문장을 return 한다. 
        return restored_text

#list to string         
def li2str(input):
    st = ""
    for i in input:
        st = st+i
    return st

#string to list
def str2li(input):
    li = []
    for i in range(len(input)):
        li.append(input[i])
    return li

#To make Jamo Dictionary
#Dictionary의 요소를 jamo로 바꾸는 함수이다. 
def makejamodict(input):
    result = []
    for i in input:
        bullet = []
        one = []
        two = []
        gre1 = tojamo(i[0])
        for j in gre1:
            for k in j:
                one.append(k)
        bullet.append(one)
        gre2 = tojamo(i[1])
        for j in gre2:
            for k in j:
                two.append(k)
        bullet.append(two)
        result.append(bullet)
    return result

#To make String dictionary
#Convert List element to String
#각 tag 사전의 key와 value가 string단위로 합쳐지게 해서 re.sub() 사용이 가능해진다. 
def makestrdict(input):
    result = []
    for i in input:
        bullet = []
        key = li2str(i[0])
        value = li2str(i[1])
        bullet.append(key)
        bullet.append(value)
        result.append(bullet)
    return result

#
EP_dict = makestrdict(EP)
EF_dict = makestrdict(EF)
NB_dict = makestrdict(NB)
JX_dict = makestrdict(JX)
XR_dict = makestrdict(XR)

Dict_list=['EP','EF','NB', 'JX','XR']
Dict_map = [EP_dict, EF_dict, NB_dict, JX_dict, XR_dict]

#divide by morpheme
#형태소로 변환 후 tag와 word를 따로 저장한다. 이때 각각의 index는 일치한다. 
def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data= mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag
    

class Changer(object):
    def __init__(self):
        print('the changer starts!')
        
    def high_low(self, stc):
        result = stc
        lis_word, lis_tag = to2lists(result)
        jam = JamoDealer(lis_word)
        for i in range(len(lis_tag)):
            for k in range(len(Dict_list)):
                #print(lis_tag[i])
                if Dict_list[k] in lis_tag[i]:
                    dic = Dict_map[Dict_list.index(Dict_list[k])]

                    res = jam.jamo[i]

                    for j in range(len(dic)):
                        res = re.sub(dic[j][0],dic[j][1],res)
                    jam.jamo[i] = res
                
        return jam.make_one()
        
    
    def processText(self,stc):
        result = stc
        res = self.high_low(result)
        spacing = Spacing()
        res = spacing(res)
        return res
    



In [6]:
import pandas as pd
number = 200
texts = pd.read_table('./data/hgu_clean.kr', sep ='\n')[:number]
texts = texts['26일이요?']

for i in range(0, number):
    tt = mec.pos(texts[i], flatten=True, join=True) 
    print(tt)

['27/SN', '일/NNBC', '은/JX', '괜찮/VA', '나요/EF', '?/SF']
['중단/NNG', '시켜서/XSV+EC', '죄송/XR', '해요/XSA+EF', './SF']
['중단/NNG', '시켜서/XSV+EC', '죄송/XR', '해요/XSA+EF', './SF']
['아/IC', ',/SC', '이런/MM', '!/SF', '도저히/MAG', '생각/NNG', '이/JKS', '안/MAG', '나/VV', '는/ETM', '걸/NNB+JKO', './SF']
['Joe/SL', "'/SY", 's/SL', 'Café/SL', '라는/VCP+ETM', '레스토랑/NNG', '에/JKB', '가/VV', '고/EC', '싶/VX', '은데/EC', ',/SC', '알/VV', '고/EC', '계세요/VX+EF', '?/SF']
['보다/MAG', '나/VA', '은/ETM', '건강/NNG', '과/JC', '생활/NNG', '회사/NNG', '의/JKG', '글렌/NNP', '토마스/NNP', '입니다/VCP+EF', './SF']
['스타워즈/NNP', '는/JX', '어때/VA+EF', '?/SF']
['데이터/NNG', '코디네이터/NNG', '나/JC', '데이터/NNG', '매니저/NNG', '가/JKS', '어떨까/VA+EC', '요/JX', '?/SF']
['화성/NNG', '에서/JKB', '온/VV+ETM', '로봇/NNG', '이/VCP', '요/EF', '!/SF', '좋/VA', '아요/EF', '!/SF', '특수/NNG', '효과/NNG', '가/JKS', '대단/XR', '하/XSA', '대요/EF', './SF']
['팩스/NNG', '를/JKO', '보내/VV+EC', '줘서/VX+EC', '감사/NNG', '합니다/XSV+EF', './SF']
['11/SN', '월/NNBC', '30/SN', '일/NNBC', '날짜/NNG', '로/JKB', '팩스/NNG', '를/JKO', '보내/VV+EC', 

In [7]:
import pandas as pd
number = 200
texts = pd.read_table('./data/hgu_clean.kr', sep ='\n')[:number]
texts = texts['26일이요?']

ch = Changer()

for i in range(0, number):
    tt = ch.processText(texts[i])
    print(tt)

the changer starts!
the jamo starts!
27일은 괜찮나?
the jamo starts!
중단시켜서 미안해.
the jamo starts!
중단시켜서 미안해.
the jamo starts!
아, 이런 !도저히 생각이 안 나는 걸.
the jamo starts!
Joe 'sCaf é라는 레스토랑에 가고 싶은데, 알고 계세?
the jamo starts!
보다 나은 건강과 생활회사의 글렌토마스인다.
the jamo starts!
스타워즈는 어때?
the jamo starts!
데이터 코디네이터나 데이터 매니저가 어떨까요?
the jamo starts!
화성에서 온 로봇이 ! 좋아! 특수효과가 대단하대.
the jamo starts!
팩스를 보내줘서 감사한다.
the jamo starts!
11월 30일 날짜로 팩스를 보내줘서 감사한다.
the jamo starts!
자격을 취득하기 위해 공부 중이에.
the jamo starts!
간호사 자격을 취득하기 위해 공부 중이에.
the jamo starts!
국가시험을 향해 공부 중이에.
the jamo starts!
공무원에 되기 위한 국가시험을 향해 공부 중이에.
the jamo starts!
잘됐어. 그럼 이제 집에 가도 되나?
the jamo starts!
미안한다. 그녀는 안에 없네. 메모 남겨드릴까?
the jamo starts!
내려줄래.
the jamo starts!
올려줄래.
the jamo starts!
작은 목소리
the jamo starts!
스미스 선생님.저화장실에 가야겠는데.
the jamo starts!
화장실이 정말 급해!
the jamo starts!
그럼 총수입의 얼마나 되는 거죠?
the jamo starts!
0136521798이에.
the jamo starts!
1, 199달러 예?
the jamo starts!
100그램에 비타민C의 함유량은 레몬보다 높다.
the jamo starts!
100미터더가서 좌회전해.
the jamo starts!
105번이그

In [9]:
txt = '어서 하세요.'
spacing = Spacing()
ch = Changer()
tt = ch.processText(txt)
print(tt)

the changer starts!
the jamo starts!
어서 하세.
